<a href="https://colab.research.google.com/github/kito2007/CV_assignment/blob/main/dsu_notice_page_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
!pip install konlpy
!pip install keybert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23767 sha256=be51515bf1bb972290fea52c253d98e648e6ffb0470f0bf3e73524f2a8334f05
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for 

In [ ]:
!pip install mysql
!pip install mysql-client
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.0-cp310-cp310-linux_x86_64.whl size=123670 sha256=ae8db9019b900aae61f6c5f582d53575b687e271ac0a83f63ba38df0520ec332
  Stored in directory: /root/.cache/pip/wheels/a4/f8/fd/0399687c0abd03c10c975ed56c692fcd3d0fb80440b5a661f1
Successfully built mysqlclient
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 36.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
import mysql.connector

# RDS 연결 설정
db_config = {
    "host": "dsu-notice-db.cipq0d6wiuy9.ap-northeast-2.rds.amazonaws.com",
    "user": "admin",
    "password": "kito2007",
}

# 데이터베이스 연결
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [ ]:
create_database_query = "CREATE DATABASE IF NOT EXISTS dsu_notice_db"
cursor.execute(create_database_query)

connection.commit()
connection.close()

In [ ]:
# RDS 연결 설정 (테이블 생성을 위해 데이터베이스 이름을 명시)
db_config = {
    "host": "dsu-notice-db.cipq0d6wiuy9.ap-northeast-2.rds.amazonaws.com",
    "user": "admin",
    "password": "kito2007",
    "database": "dsu_notice_db",
}

# 데이터베이스 연결
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# 테이블 생성 쿼리 실행
create_table_query = """
CREATE TABLE IF NOT EXISTS notices (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    content TEXT,
    keyword1 VARCHAR(50),
    keyword2 VARCHAR(50),
    keyword3 VARCHAR(50),
    keyword4 VARCHAR(50),
    keyword5 VARCHAR(50)
)
"""
cursor.execute(create_table_query)

connection.commit()
connection.close()

In [ ]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.dongseo.ac.kr/kr/?pCode=MN2000194"
page = 1

# 고유식별자 목록을 저장할 리스트
idx_list = []

while page <= 10:
    # 각 페이지에 접속하여 HTML 코드 가져오기
    url = f"{base_url}&pg={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # 공지사항 테이블의 각 row에서 "idx" 파라미터 값을 추출하여 리스트에 추가
    for row in soup.find_all("tr", class_=["isnotice", "child_1", "child_2"]):
        link = row.find("a", href=True)
        if link:
            idx = link["href"].split("&idx=")[-1]
            idx_list.append(idx)

    # 다음 페이지로 이동
    page += 1

# 추출한 고유식별자 목록 출력
print(idx_list)


['92988', '92936', '92881', '92838', '93041', '93024', '92988', '92976', '92936', '92906', '92881', '92867', '92838', '92814', '92797', '92783', '92751', '92744', '92718', '92988', '92936', '92881', '92838', '92716', '92700', '92687', '92684', '92639', '92614', '92610', '92425', '92421', '92419', '92404', '92382', '92378', '92370', '92361', '92988', '92936', '92881', '92838', '92344', '92342', '92332', '92308', '92286', '92232', '92223', '92194', '92166', '92097', '91918', '91917', '91926', '91910', '91814', '92988', '92936', '92881', '92838', '91797', '91733', '91730', '91704', '91701', '91692', '91647', '91632', '91625', '91606', '91603', '91569', '91537', '91499', '91498', '92988', '92936', '92881', '92838', '91449', '91275', '91421', '91412', '91367', '91360', '91345', '91331', '91328', '91276', '91214', '91203', '91178', '91177', '91175', '92988', '92936', '92881', '92838', '91128', '91043', '90864', '90853', '90842', '90829', '90828', '90819', '90794', '90770', '90769', '90765', 

In [ ]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from keybert import KeyBERT
from collections import Counter

# 게시글 내용 추출 함수
def extract_content_from_link(link):
    response = requests.get(link)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")

    # 게시글 내용 추출
    content_tag = soup.find("div", class_="board-view-cont")
    if content_tag is not None:
        content = content_tag.get_text()
    else:
        content = ""

    return content

# 게시글 링크 리스트
links = []
for num in idx_list:
  links.append("https://www.dongseo.ac.kr/kr/?pCode=MN2000194&mode=view&idx="+num)


from collections import Counter
from konlpy.tag import Okt

# 불용어(Stopwords) 목록
stopwords = ['현재', '페이지']

# Okt 객체 생성
okt = Okt()

cnt = 0
# 각 게시글의 내용 추출 및 형태소 분석
for link in links:
    content = extract_content_from_link(link)
    # 텍스트 전처리 등 추가적인 전처리 작업을 수행할 수 있습니다.

    # 형태소 분석
    morphemes = okt.pos(content, stem=True)

    # 명사 추출 및 불용어 제외
    nouns = [word for word, pos in morphemes if pos in ['Noun'] and word not in stopwords]

    # 명사들의 빈도수를 세기 위해 Counter 사용
    nouns_counter = Counter(nouns)

    # 상위 5개의 가장 빈도가 높은 명사를 추출
    top_keywords = nouns_counter.most_common(5)

    print("idx: " + idx_list[cnt])
    cnt = cnt + 1
    print("텍스트에서 가장 중요한 단어 5개:")
    for keyword, count in top_keywords:
        print(f"{keyword}: {count}회")

    print("--------------------------------------")


idx: 92988
텍스트에서 가장 중요한 단어 5개:
수강신청: 16회
수강: 8회
학년: 7회
공유: 7회
예비: 5회
--------------------------------------
idx: 92936
텍스트에서 가장 중요한 단어 5개:
공유: 7회
휴가: 3회
집중: 2회
기간: 2회
월: 2회
--------------------------------------
idx: 92881
텍스트에서 가장 중요한 단어 5개:
신청: 20회
복학: 19회
학기: 8회
공유: 7회
기간: 6회
--------------------------------------
idx: 92838
텍스트에서 가장 중요한 단어 5개:
강의: 19회
평가: 17회
과목: 10회
공유: 7회
실습: 5회
--------------------------------------
idx: 93041
텍스트에서 가장 중요한 단어 5개:
공유: 7회
취업: 3회
계: 3회
교육: 3회
및: 3회
--------------------------------------
idx: 93024
텍스트에서 가장 중요한 단어 5개:
납부: 44회
등록금: 31회
신청: 27회
등록: 21회
학점: 21회
--------------------------------------
idx: 92988
텍스트에서 가장 중요한 단어 5개:
수강신청: 16회
수강: 8회
학년: 7회
공유: 7회
예비: 5회
--------------------------------------
idx: 92976
텍스트에서 가장 중요한 단어 5개:
학습: 12회
자기: 11회
주도: 10회
경험: 10회
학점: 9회
--------------------------------------
idx: 92936
텍스트에서 가장 중요한 단어 5개:
공유: 7회
휴가: 3회
집중: 2회
기간: 2회
월: 2회
--------------------------------------
idx: 92906
텍스트에서 가장 중요한 단어 5개:
학점: 12회

In [ ]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from keybert import KeyBERT
from collections import Counter

# 게시글 내용 추출 함수
def extract_content_from_link(link):
    response = requests.get(link)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")

    # 게시글 내용 추출
    content_tag = soup.find("div", class_="board-view-cont")
    if content_tag is not None:
        content = content_tag.get_text()
    else:
        content = ""

    # 게시글 날짜 추출
    date_tag = soup.find("span", class_="board-view-date")
    if date_tag is not None:
        date = date_tag.get_text()
    else:
        date = ""

    return content, date

# 게시글 링크 리스트
links = []
for num in idx_list:
    links.append("https://www.dongseo.ac.kr/kr/?pCode=MN2000194&mode=view&idx=" + num)

# 불용어(Stopwords) 목록
stopwords = ['현재', '페이지']

# Okt 객체 생성
okt = Okt()

cnt = 0
# 각 게시글의 내용 추출 및 형태소 분석
for link in links:
    content, date = extract_content_from_link(link)
    # 텍스트 전처리 등 추가적인 전처리 작업을 수행할 수 있습니다.

    # 형태소 분석
    morphemes = okt.pos(content, stem=True)

    # 명사 추출 및 불용어 제외
    nouns = [word for word, pos in morphemes if pos in ['Noun'] and word not in stopwords]

    # 명사들의 빈도수를 세기 위해 Counter 사용
    nouns_counter = Counter(nouns)

    # 상위 5개의 가장 빈도가 높은 명사를 추출
    top_keywords = nouns_counter.most_common(5)

    print("idx: " + idx_list[cnt])
    cnt = cnt + 1
    print("텍스트에서 가장 중요한 단어 5개:")
    for keyword, count in top_keywords:
        print(f"{keyword}: {count}회")

    print("게시물 날짜:", date)
    print("--------------------------------------")


idx: 92988
텍스트에서 가장 중요한 단어 5개:
수강신청: 16회
수강: 8회
학년: 7회
공유: 7회
예비: 5회
게시물 날짜: 
--------------------------------------
idx: 92936
텍스트에서 가장 중요한 단어 5개:
공유: 7회
휴가: 3회
집중: 2회
기간: 2회
월: 2회
게시물 날짜: 
--------------------------------------
idx: 92881
텍스트에서 가장 중요한 단어 5개:
신청: 20회
복학: 19회
학기: 8회
공유: 7회
기간: 6회
게시물 날짜: 
--------------------------------------
idx: 92838
텍스트에서 가장 중요한 단어 5개:
강의: 19회
평가: 17회
과목: 10회
공유: 7회
실습: 5회
게시물 날짜: 
--------------------------------------
idx: 93041
텍스트에서 가장 중요한 단어 5개:
공유: 7회
취업: 3회
계: 3회
교육: 3회
및: 3회
게시물 날짜: 
--------------------------------------
idx: 93024
텍스트에서 가장 중요한 단어 5개:
납부: 44회
등록금: 31회
신청: 27회
등록: 21회
학점: 21회
게시물 날짜: 
--------------------------------------
idx: 92988
텍스트에서 가장 중요한 단어 5개:
수강신청: 16회
수강: 8회
학년: 7회
공유: 7회
예비: 5회
게시물 날짜: 
--------------------------------------
idx: 92976
텍스트에서 가장 중요한 단어 5개:
학습: 12회
자기: 11회
주도: 10회
경험: 10회
학점: 9회
게시물 날짜: 
--------------------------------------
idx: 92936
텍스트에서 가장 중요한 단어 5개:
공유: 7회
휴가: 3회
집중: 2회
기간: 2회
월: 2회
게시물 날